In [21]:
import pandas as pd
import numpy as np
from geopy.distance import vincenty
import cPickle as pickle


# importing all datasets
weather_path = '../assets/weather.csv'
spray_path = '../assets/spray.csv'
train_path = '../assets/train.csv'
weather_df = pd.read_csv(weather_path)
spray_df = pd.read_csv(spray_path)
train_df = pd.read_csv(train_path)

# From Kaggle description page:
# for the weather data, here are the GEO coordinates for the two stations (two airports)
# Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
# Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level
LAT1 = 41.995
LON1 = -87.933
LAT2 = 41.786
LON2 = -87.752
L1 = (LAT1,LON1)
L2 = (LAT2,LON2)


# looking only for these particular days (in training set)
training_days = train_df.Date.unique()
print 'unique days in training set', len(train_df.Date.unique())
print 'unique days in weather data',len(weather_df.Date.unique())

# only need weather data for the day that was recorded, subsetting for efficiency
weather_sub = weather_df[weather_df['Date'].isin(training_days)]
print 'total weather row count:', len(weather_df),' only relevant days:', len(weather_sub)



# adding the fields, 
# initally taking the temperature of the closer airport. Don't expect the temperature
# to drastically change between the two airports. 
# will use an apply function to add several columns, distance to each airport
# and the rough distance %
# and the closer airport (more dominant)

def makeAirportPct(x):
    current = (x['Latitude'],x['Longitude']  )
    dist_to_stat1 = vincenty(current,L1).miles
    dist_to_stat2 = vincenty(current,L2).miles
    denom = dist_to_stat1 + dist_to_stat2
    x['STATION1DIST'] = dist_to_stat1
    x['STATION2DIST'] = dist_to_stat2
    x['STATION1PCT'] = dist_to_stat1 / denom
    x['STATION2PCT'] = dist_to_stat2 / denom
    if dist_to_stat1 < dist_to_stat2:
        x['CLOSER_STATION'] = 1
    else:
        x['CLOSER_STATION'] = 2
    return x

train_df = train_df.apply(makeAirportPct,axis=1)


#lastly will merge the weather in, will match by DATE and STATION
train_df = train_df.merge(weather_sub,how='left',left_on=['Date','CLOSER_STATION'], right_on=['Date','Station'])


unique days in training set 95
unique days in weather data 1472
total weather row count: 2944  only relevant days: 190


In [22]:
#===============================================================================================================
# Exporting into a pickle to be picked up for other uses
#===============================================================================================================

with open('train_w_weather.p','wb') as f:
    pickle.dump(train_df,f)

In [23]:
#===============================================================================================================
# Additional exploration below
#===============================================================================================================



In [24]:
train_df.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5


In [25]:
'''
Species
Block
Date
Trap
Tmax
Tmin
Tavg
Dewpoint
WetBulb
Heat
Cool
'''

train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10506 entries, 0 to 10505
Data columns (total 38 columns):
Date                      10506 non-null object
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
STATION1DIST              10506 non-null float64
STATION2DIST              10506 non-null float64
STATION1PCT               10506 non-null float64
STATION2PCT               10506 non-null float64
CLOSER_STATION            10506 non-null int64
Station                   10506 non-null int64
Tmax                      1

In [26]:
train_df.head(2).transpose()

,0,1
Date,2007-05-29,2007-05-29
Address,"4100 North Oak Park Avenue, Chicago, IL 60634,...","4100 North Oak Park Avenue, Chicago, IL 60634,..."
Species,CULEX PIPIENS/RESTUANS,CULEX RESTUANS
Block,41,41
Street,N OAK PARK AVE,N OAK PARK AVE
Trap,T002,T002
AddressNumberAndStreet,"4100 N OAK PARK AVE, Chicago, IL","4100 N OAK PARK AVE, Chicago, IL"
Latitude,41.9547,41.9547
Longitude,-87.801,-87.801
AddressAccuracy,9,9


In [27]:
train_df.Sunset.unique()

array(['1917', '-', '1923', '1931', '1928', '1924', '1911', '1904', '1902',
       '1854', '1845', '1840', '1822', '1809', '1756', '1755', '1746',
       '1729', '1720', '1916', '1920', '1921', '1927', '1930', '1929',
       '1925', '1919', '1913', '1856', '1839', '1835', '1824', '1805',
       '1758', '1744', '1734', '1926', '1918', '1915', '1907', '1858',
       '1848', '1837', '1827', '1826', '1760', '1748', '1735', '1903',
       '1843', '1832', '1819', '1742'], dtype=object)

In [28]:
train_df.Water1.unique()

array(['M'], dtype=object)

In [29]:
train_df.CodeSum.unique()

array(['BR HZ', 'RA BR', 'TSRA BR HZ VCTS', ' ', 'TSRA RA BR VCTS',
       'TS TSRA RA BR HZ VCTS', 'DZ BR', 'RA BR HZ', 'HZ', 'VCTS',
       'TSRA RA DZ BR HZ', 'TSRA RA', 'TSRA BR HZ', 'TSRA RA BR',
       'TS TSRA RA BR', 'BR', 'TSRA HZ', 'TSRA', 'RA', 'TS TSRA BR',
       'RA DZ BR', 'TS TSRA BR HZ', 'TS TSRA RA VCTS', 'TSRA BR', 'TS BR',
       'TS', 'FG BR HZ', 'TS TSRA RA FG BR HZ'], dtype=object)

In [30]:
train_df.WnvPresent.value_counts()

0    9955
1     551
Name: WnvPresent, dtype: int64

In [32]:
for x in train_df.columns:
    print x

Date
Address
Species
Block
Street
Trap
AddressNumberAndStreet
Latitude
Longitude
AddressAccuracy
NumMosquitos
WnvPresent
STATION1DIST
STATION2DIST
STATION1PCT
STATION2PCT
CLOSER_STATION
Station
Tmax
Tmin
Tavg
Depart
DewPoint
WetBulb
Heat
Cool
Sunrise
Sunset
CodeSum
Depth
Water1
SnowFall
PrecipTotal
StnPressure
SeaLevel
ResultSpeed
ResultDir
AvgSpeed
